<a href="https://colab.research.google.com/github/jhackisneros/VISION_ARTIFICIAL/blob/main/colab_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# VISION_ARTIFICIAL - Notebook Final Integrado

🔹 Importar librerías

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from skimage.util import random_noise
from skimage import exposure
import os


🔹 Configuración de reproducibilidad

In [ ]:
tf.random.set_seed(42)
np.random.seed(42)
try:
    tf.config.experimental.enable_op_determinism(True)
except:
    pass


🔹 Crear carpetas necesarias

In [ ]:
os.makedirs("../data/processed", exist_ok=True)
os.makedirs("../models/ensemble", exist_ok=True)
os.makedirs("../experiments", exist_ok=True)


# CARGA Y PREPROCESAMIENTO

In [ ]:
from tensorflow.keras.datasets import cifar10

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype("float32")/255.0
x_test  = x_test.astype("float32")/255.0
y_train = to_categorical(y_train, 10)
y_test  = to_categorical(y_test, 10)

# Guardar procesados para otros notebooks

In [ ]:
np.save("../data/processed/x_train.npy", x_train)
np.save("../data/processed/y_train.npy", y_train)
np.save("../data/processed/x_test.npy", x_test)
np.save("../data/processed/y_test.npy", y_test)

# CONSTRUCCIÓN CNN

In [ ]:
cnn = Sequential([
    Conv2D(32, (3,3), activation='relu', padding='same', input_shape=(32,32,3)),
    MaxPooling2D((2,2)),
    Conv2D(64, (3,3), activation='relu', padding='same'),
    MaxPooling2D((2,2)),
    Flatten(),
    Dense(64, activation='relu'),
    Dense(10, activation='softmax')
])

cnn.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

 # ENTRENAMIENTO

In [ ]:
history = cnn.fit(x_train, y_train, validation_data=(x_test, y_test),
                  epochs=10, batch_size=64)

# Guardar modelo
cnn.save("../models/cnn_best.keras")

# ENSEMBLE SIMPLIFICADO

In [ ]:
# Para ejemplo final, clonamos el mismo modelo 3 veces
ensemble_models = [load_model("../models/cnn_best.keras") for _ in range(3)]

def ensemble_predict(models, x):
    pred_probs = np.zeros((x.shape[0], 10))
    for model in models:
        pred_probs += model.predict(x)
    pred_probs /= len(models)
    return np.argmax(pred_probs, axis=1)

y_pred_ensemble = ensemble_predict(ensemble_models, x_test)
acc_ensemble = accuracy_score(np.argmax(y_test, axis=1), y_pred_ensemble)
print(f"✅ Accuracy ensemble: {acc_ensemble*100:.2f}%")


# ROBUSTEZ

In [ ]:
# Ruido gaussiano
x_test_noisy = np.array([random_noise(img, mode='gaussian', var=0.02) for img in x_test])
acc_noisy = accuracy_score(np.argmax(y_test, axis=1), ensemble_predict(ensemble_models, x_test_noisy))

# Cambio de brillo
x_test_bright = np.array([exposure.adjust_gamma(img, gamma=0.5) for img in x_test])
acc_bright = accuracy_score(np.argmax(y_test, axis=1), ensemble_predict(ensemble_models, x_test_bright))

print(f"✅ Accuracy con ruido: {acc_noisy*100:.2f}%")
print(f"✅ Accuracy con brillo: {acc_bright*100:.2f}%")

# EXPLAINABILITY - GRAD-CAM

In [ ]:
def get_gradcam(img_array, model, layer_name="conv2d_1"):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        class_idx = np.argmax(predictions[0])
        loss = predictions[:, class_idx]

    grads = tape.gradient(loss, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = np.maximum(heatmap, 0) / (np.max(heatmap) + 1e-8)
    return heatmap, class_idx

def display_gradcam(img, heatmap, alpha=0.4):
    import cv2
    heatmap = cv2.resize(heatmap.numpy(), (img.shape[1], img.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * alpha + np.uint8(img*255)
    plt.imshow(superimposed_img.astype(np.uint8))
    plt.axis('off')
    plt.show()

    # Ejemplo Grad-CAM
img = x_test[0][np.newaxis,...]
heatmap, class_idx = get_gradcam(img, cnn, layer_name="conv2d_1")
print(f"Predicción clase: {class_idx}")
display_gradcam(x_test[0], heatmap)

NameError: name 'x_test' is not defined